# OpenCV: Feature Detection and Description
## Harris Corner Detection

### Goal

In this chapter,

- We will understand the concepts behind Harris Corner Detection.
- We will see the functions: cv2.cornerHarris(), cv2.cornerSubPix()

### Theory

In last chapter, we saw that corners are regions in the image with large variation in intensity in all the directions. One early attempt to find these corners was done by Chris Harris & Mike Stephens in their paper *A Combined Corner and Edge Detector* in 1988, so now it is called Harris Corner Detector. He took this simple idea to a mathematical form. It basically finds the difference in intensity for a displacement of $(u,v)$ in all directions. This is expressed as below

![Image](https://opencv-python-tutroals.readthedocs.io/en/latest/_images/math/1948f2850c912ce3394b61ba99b546f1bf7a6adc.png)

Window function is either a rectangular window or gaussian window which gives weights to pixels underneath.

We have to maximize this function $E(u,v)$ for corner detection. That means, we have to maximize the second term. Applying Taylor Expansion to above equation and using some mathematical steps (please refer any standard text books you like for full derivation), we get the final equation as:

![Image](https://opencv-python-tutroals.readthedocs.io/en/latest/_images/math/3aea48f44ebcc36dda8055c842e209d8886d7a80.png)

where

![Image](https://opencv-python-tutroals.readthedocs.io/en/latest/_images/math/a33b6840bcc7c240425581fe5e39e58d868f9893.png)

Here, $I_x$ and $I_y$ are image derivatives in $x$ and $y$ directions respectively. (Can be easily found out using **cv2.Sobel()**).

Then comes the main part. After this, they created a score, basically an equation, which will determine if a window can contain a corner or not.

![Image](https://opencv-python-tutroals.readthedocs.io/en/latest/_images/math/8499c5fd1fb4572f17432043824392ba87e027e6.png)

where

- $det(M)=\lambda_1\lambda_2$
- $trace(M)=\lambda_1+\lambda_2$
- $\lambda_1$ and $\lambda_2$ are the eigen values of $M$

So the values of these eigen values decide whether a region is corner, edge or flat.

- When $|R|$ is small, which happens when $\lambda_1$ and $\lambda_2$ are small, the region is flat.
- When $R<0$, which happens when $\lambda_1 \gg \lambda_2$ or vice versa, the region is edge.
- When $R$ is large, which happens when $\lambda_1$ and $\lambda_2$ are large and $\lambda_1 \sim \lambda_2$, the region is a corner.

It can be represented in a nice picture as follows:

![Image](https://opencv-python-tutroals.readthedocs.io/en/latest/_images/harris_region.jpg)

So the result of Harris Corner Detection is a grayscale image with these scores. Thresholding for a suitable give you the corners in the image. We will do it with a simple image.

### Harris Corner Detector in OpenCV
OpenCV has the function *cv2.cornerHarris()* for this purpose. Its arguments are :

- **img** - Input image, it should be grayscale and float32 type.
- **blockSize** - It is the size of neighbourhood considered for corner detection
- **ksize** - Aperture parameter of Sobel derivative used.
- **k** - Harris detector free parameter in the equation.

See the example below:

In [3]:
import cv2
import numpy as np

filename = 'Images/chessboard.jpg'
img = cv2.imread(filename)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)

#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]

cv2.imshow('dst',img)
if cv2.waitKey(0) & 0xFF == ord('q'):
    cv2.destroyAllWindows()

In [ ]:
# convert to gray image
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)

In [ ]:
#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)

In [ ]:
# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]

In [ ]:
cv2.imshow('dst',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

### Corner with SubPixel Accuracy

In [ ]:
import cv2
import numpy as np

In [ ]:
filename = 'Images/chessboard.jpg'
img = cv2.imread(filename)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [ ]:
# find Harris corners
gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)
dst = cv2.dilate(dst,None)
ret, dst = cv2.threshold(dst,0.01*dst.max(),255,0)
dst = np.uint8(dst)

In [ ]:
# find centroids
ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

In [ ]:
# define the criteria to stop and refine the corners
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)

In [ ]:
# Now draw them
res = np.hstack((centroids,corners))
res = np.int0(res)
img[res[:,1],res[:,0]]=[0,0,255]
img[res[:,3],res[:,2]] = [0,255,0]
cv2.imwrite('subpixel5.png',img)

## Shi-Tomasi Corner Detector & Good Features to Track

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt


In [ ]:
img = cv2.imread('Images/blox.jpg')
print(img.shape)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [ ]:
corners = cv2.goodFeaturesToTrack(gray,2,0.11,1)
corners = np.int0(corners)
for i in corners:
    x,y = i.ravel()
    cv2.circle(img,(x,y),3,255,-1)

plt.imshow(img)
plt.axis('off')
plt.show()